In [ ]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
# Login to Weights & Biases
wandb.login(key="your_api_key_here")

# Use DistilBERT tokenizer and model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import optuna
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Load and preprocess the dataset
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_CDs_and_Vinyl", trust_remote_code=True)
shuffled_dataset = dataset["full"].shuffle(seed=42)
subset_size = int(1 * len(shuffled_dataset))
subset_dataset = shuffled_dataset.select(range(subset_size))

# Filter out samples where rating == 3
subset_dataset = subset_dataset.filter(lambda x: x["rating"] != 3)

# Initialize the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize_and_format(examples):
    # Tokenize and encode the text
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    # Create binary labels
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Apply the tokenizer to the dataset
tokenized_dataset = subset_dataset.map(tokenize_and_format, batched=True)

# Save the processed dataset
tokenized_dataset.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")

# Split the dataset into train/test sets
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

CDs_and_Vinyl.jsonl:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/4543582 [00:00<?, ? examples/s]

Saving the dataset (0/17 shards):   0%|          | 0/4543582 [00:00<?, ? examples/s]

In [ ]:
# Step 2': Reload dataset if needed in new runtime
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
# Step 3A: Hyperparameter search and Trainer setup

# Model initialization using DistilBERT
def model_init():
    return DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Evaluation metric computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Optuna hyperparameter space
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    }

# Initial training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=96,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=8,
    warmup_steps=500,
    report_to=["wandb"],
    run_name="distilbert_amazon_reviews",
    logging_steps=50,
    logging_first_step=True
)

# Sample a small subset for demonstration
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Trainer setup with DistilBERT
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Run hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=10,
    hp_space=hp_space,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-04-02 07:26:21,641] A new study created in memory with name: no-name-9110f4e8-9e5d-4f4b-89d7-b20ea6a0b564
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.224700,0.205091,0.933957,0.902063,0.872276,0.933957
2,0.140200,0.124579,0.953220,0.953129,0.953039,0.953220
3,0.127600,0.134161,0.955971,0.952790,0.951739,0.955971
4,0.074300,0.137046,0.958173,0.956777,0.955869,0.958173


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.153700,0.140232,0.954320,0.952271,0.951051,0.954320
2,0.108300,0.181975,0.947166,0.949865,0.953773,0.947166


[I 2025-04-02 07:49:20,569] Trial 8 pruned. 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,█▁
eval/f1,█▁
eval/loss,▁█
eval/precision,▁█
eval/recall,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▃▂▁▂▁▁▁▅▁▃▁▄▁▃▄▂▆█▁▁▁▁▄▁▂▁▁▁▁▁▁▁▁█▁▁▁


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.144500,0.131056,0.955421,0.952082,0.951009,0.955421
2,0.136400,0.120305,0.957622,0.958179,0.958817,0.957622
3,0.094500,0.136556,0.962576,0.961493,0.960798,0.962576
4,0.032600,0.154774,0.962025,0.961951,0.961879,0.962025


[I 2025-04-02 07:51:44,434] Trial 9 finished with value: 3.847881512789253 and parameters: {'learning_rate': 7.241659439654104e-06, 'per_device_train_batch_size': 16}. Best is trial 7 with value: 3.8561066253126226.


In [ ]:
# Step 3B: Final training with best hyperparameters
print("Best run hyperparameters:", best_run.hyperparameters)

# Update training arguments with best hyperparameters
updated_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_run.hyperparameters["learning_rate"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to=["wandb"],
    run_name="distilbert_amazon_reviews",
    logging_steps=50
)

# Reload training and eval subsets
train_subset_final = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset_final = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Final trainer instance
final_trainer = Trainer(
    model_init=model_init,
    args=updated_training_args,
    train_dataset=train_subset_final,
    eval_dataset=eval_subset_final,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


Best run hyperparameters: {'learning_rate': 3.51204355566076e-05, 'per_device_train_batch_size': 8}


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 4: Start final training and save model
train_result = final_trainer.train()
final_trainer.save_model("/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_CDs_distilbert")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.120300,0.145074,0.957622,0.953752,0.953305,0.957622
2,0.075500,0.169457,0.959274,0.958787,0.958366,0.959274
3,0.034000,0.217665,0.959824,0.957830,0.956915,0.959824
